## <u>Dart Tab Fusion</u>  

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-4.1.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-4.1.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-4.1.0-py3-none-any.whl


In [2]:
import numpy as np
import polars as pl
import pandas as pd
from pathlib import Path
import json
import os 
import joblib
from datetime import datetime 
from sklearn.metrics import mean_squared_error as mse
import kaggle_evaluation.mcts_inference_server 

import matplotlib.pyplot as plt
class CFG:
    # Paths to competition data
    train_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
    test_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')
    subm_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv')
    # Feature engineering (FE) arguments
    batch_size = 16384*2
    low_memory = True
    top_features= True
    drop_gameruleset=True
    version = "dart_5f_V8"
    path =  "/kaggle/input/mcts-gaurav-dart-tab/"#"/kaggle/working/"#
    infer_path = "/kaggle/input/mcts-gaurav-dart-tab/dart_5f_V8"
    # Color for EDA and MD
    color = '#E0BFB8' 
    early_stop = 50
    n_features=300
    n_splits = 8
    var_threshold = 0.01#0.005#0.01
    log_steps = 600
    tfidf = False
    DEBUG = False
    single_run = True
    seed=2024
    TRAIN=False
    response_variate='utility_agent1'
    # LightGBM parameters
    lgb_p = {
        'objective': 'regression',
        'boosting_type':'dart',
        'n_estimators': 12000,
        'learning_rate': 0.075,
        'extra_trees': True,
        'reg_lambda': 0.8,
        'colsample_bytree': 0.6,#0.6 
        'subsample': 0.8, 
        'num_leaves': 64,
        'metric': 'rmse',
        'device': 'cpu',
        #'max_depth': 4,
        'max_bin': 128,#128,
        'verbose': -1,
        'seed': 42,
        'num_threads':os.cpu_count(),
    } 

if not os.path.exists(f"{CFG.path}/{CFG.version}"):
    os.makedirs(f"{CFG.path}/{CFG.version}")   

In [3]:
train_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
test_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')
subm_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv') 

# TRAIN INFER WITH SCRIPTS

In [4]:
# import glob
# import shutil
# import os

# # Assuming CFG.infer_path is the directory containing the files
# source_path = CFG.infer_path
# destination_path = f"{CFG.path}/{CFG.version}/"

# # Create destination directory if it doesn't exist
# os.makedirs(destination_path, exist_ok=True)

# # Copy each file from source to destination
# for file in glob.glob(os.path.join(source_path, '*')):
#     shutil.copy(file, destination_path)

In [5]:
# %%bash
# for file in /kaggle/working/dart_5f_V8/tabnet*.mcin; do
#     mv "$file" "${file%.mcin}"
# done

In [6]:
import sys
sys.path.insert(0,f'/kaggle/input/train-infer-modifed') 
from train_infer import MD
sys.path.insert(0,f'{CFG.path}/{CFG.version}/')  
from fe import FE


In [7]:
top_features = pd.read_csv("/kaggle/input/mcts-models/dart_5f_V7/lgbm_fe.csv")["name"][:CFG.n_features].to_list()
fe = FE(CFG.batch_size, CFG.low_memory,CFG,top_features)
md = MD(CFG.early_stop, CFG.n_splits,CFG.color,CFG.lgb_p,CFG,top_features)    

def train_model(w):  
    global cat_cols, lgb_models,final_model,drop_cols, lgb_oof_preds,tab_models,lgb_wof_oofs,tab_oofs,cat_models, cat_oofs
    lgb_models_oof=None
    processed_path = f"{CFG.path}/{CFG.version}_processed_train_data.csv"
    cat_cols_path = f"{CFG.path}/{CFG.version}_cat_cols.json"

    if os.path.exists(processed_path+"1"):
        print("Loading processed training data...")
        train = pl.read_csv(processed_path)
        with open(cat_cols_path, 'r') as f:
            cat_cols = json.load(f)
    else:
        print("Processing raw training data...")
        df = pl.read_csv(CFG.train_path, low_memory=CFG.low_memory, batch_size=CFG.batch_size)  
        df = fe.process_agent_cols(df) 
        df = fe.set_datatypes(df)  
        drop_cols = fe.remove_low_variance_features(df, exclude_cols=['utility_agent1','p1_selection','p1_exploration','p2_exploration','p1_playout','p1_bounds','p2_selection','p2_playout','p2_bounds' ])
        train, cat_cols = fe.process_data(df, drop_cols) 
#         train.write_csv(processed_path)
#         with open(cat_cols_path, 'w') as f:
#             json.dump(cat_cols, f)
        print(f"Processed data saved to {processed_path} and cat_cols saved to {cat_cols_path}")
        
    print("#### Columns used ",train.shape, cat_cols)
    cat_models, cat_oofs = md.train_cat(train, cat_cols)      
    lgb_models, lgb_oof_preds,fcls = md.train_lgb(train, cat_cols, '')
    #md.feature_importances(lgb_models,fcls)
    if not CFG.single_run:
        train = train.with_columns(pl.Series(lgb_oof_preds).alias('lgb_oof_preds')) 
        lgb_models_oof, _,_ = md.train_lgb(train, cat_cols, title='_wOOF')
    #md.feature_importances(lgb_models,fcls)
    #train = train.with_columns(pl.Series(lgb_oof_preds).alias('lgb_oof_preds')) 
    #lgb_models_oof, lgb_wof_oofs, _ = md.train_lgb(train, cat_cols, title='LightGBM_wOOF') 
    #tab_models, tab_oofs = md.train_tab(train, cat_cols)      
    score = mse(train[CFG.response_variate],(lgb_oof_preds*w[0]+cat_oofs*w[1]), squared=False)
    print(f"####  Ensemble Score {score} ####")

# Catboost

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)


#Reduce Memory Usage
def reduce_memory_usage(df):
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            # else:
            #     df[col] = df[col].astype('category')
    
    return df

ignore_constant_columns =  ['Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating', 'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules', 'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire', 'TwoPlayer', 'Multiplayer', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 'Container', 'Board', 'PrismShape', 'ParallelogramShape', 'RectanglePyramidalShape', 'TargetShape', 'BrickTiling', 'CelticTiling', 'QuadHexTiling', 'Hints', 'PlayableSites', 'Component', 'DiceD3', 'BiasedDice', 'Card', 'Domino', 'Rules', 'SituationalTurnKo', 'SituationalSuperko', 'InitialAmount', 'InitialPot', 'Play', 'BetDecision', 'BetDecisionFrequency', 'VoteDecisionFrequency', 'ChooseTrumpSuitDecision', 'ChooseTrumpSuitDecisionFrequency', 'LeapDecisionToFriend', 'LeapDecisionToFriendFrequency', 'HopDecisionEnemyToFriend', 'HopDecisionEnemyToFriendFrequency', 'HopDecisionFriendToFriend', 'FromToDecisionWithinBoard', 'FromToDecisionBetweenContainers', 'BetEffect', 'BetEffectFrequency', 'VoteEffectFrequency', 'SwapPlayersEffectFrequency', 'TakeControl', 'TakeControlFrequency', 'PassEffectFrequency', 'SetCost', 'SetCostFrequency', 'SetPhase', 'SetPhaseFrequency', 'SetTrumpSuit', 'SetTrumpSuitFrequency', 'StepEffectFrequency', 'SlideEffectFrequency', 'LeapEffectFrequency', 'HopEffectFrequency', 'FromToEffectFrequency', 'SwapPiecesEffect', 'SwapPiecesEffectFrequency', 'ShootEffect', 'ShootEffectFrequency', 'MaxCapture', 'OffDiagonalDirection', 'Information', 'HidePieceType', 'HidePieceOwner', 'HidePieceCount', 'HidePieceRotation', 'HidePieceValue', 'HidePieceState', 'InvisiblePiece', 'End', 'LineDrawFrequency', 'ConnectionDraw', 'ConnectionDrawFrequency', 'GroupLossFrequency', 'GroupDrawFrequency', 'LoopLossFrequency', 'LoopDraw', 'LoopDrawFrequency', 'PatternLoss', 'PatternLossFrequency', 'PatternDraw', 'PatternDrawFrequency', 'PathExtentEndFrequency', 'PathExtentWinFrequency', 'PathExtentLossFrequency', 'PathExtentDraw', 'PathExtentDrawFrequency', 'TerritoryLoss', 'TerritoryLossFrequency', 'TerritoryDraw', 'TerritoryDrawFrequency', 'CheckmateLoss', 'CheckmateLossFrequency', 'CheckmateDraw', 'CheckmateDrawFrequency', 'NoTargetPieceLoss', 'NoTargetPieceLossFrequency', 'NoTargetPieceDraw', 'NoTargetPieceDrawFrequency', 'NoOwnPiecesDraw', 'NoOwnPiecesDrawFrequency', 'FillLoss', 'FillLossFrequency', 'FillDraw', 'FillDrawFrequency', 'ScoringDrawFrequency', 'NoProgressWin', 'NoProgressWinFrequency', 'NoProgressLoss', 'NoProgressLossFrequency', 'SolvedEnd', 'PositionalRepetition', 'SituationalRepetition', 'Narrowness', 'Variance', 'DecisivenessMoves', 'DecisivenessThreshold', 'LeadChange', 'Stability', 'DramaAverage', 'DramaMedian', 'DramaMaximum', 'DramaMinimum', 'DramaVariance', 'DramaChangeAverage', 'DramaChangeSign', 'DramaChangeLineBestFit', 'DramaChangeNumTimes', 'DramaMaxIncrease', 'DramaMaxDecrease', 'MoveEvaluationAverage', 'MoveEvaluationMedian', 'MoveEvaluationMaximum', 'MoveEvaluationMinimum', 'MoveEvaluationVariance', 'MoveEvaluationChangeAverage', 'MoveEvaluationChangeSign', 'MoveEvaluationChangeLineBestFit', 'MoveEvaluationChangeNumTimes', 'MoveEvaluationMaxIncrease', 'MoveEvaluationMaxDecrease', 'StateEvaluationDifferenceAverage', 'StateEvaluationDifferenceMedian', 'StateEvaluationDifferenceMaximum', 'StateEvaluationDifferenceMinimum', 'StateEvaluationDifferenceVariance', 'StateEvaluationDifferenceChangeAverage', 'StateEvaluationDifferenceChangeSign', 'StateEvaluationDifferenceChangeLineBestFit', 'StateEvaluationDifferenceChangeNumTimes', 'StateEvaluationDifferenceMaxIncrease', 'StateEvaluationDifferenceMaxDecrease', 'BoardSitesOccupiedMinimum', 'BranchingFactorMinimum', 'DecisionFactorMinimum', 'MoveDistanceMinimum', 'PieceNumberMinimum', 'ScoreDifferenceMinimum', 'ScoreDifferenceChangeNumTimes', 'Roots', 'Cosine', 'Sine', 'Tangent', 'Exponential', 'Logarithm', 'ExclusiveDisjunction', 'Float', 'HandComponent', 'SetHidden', 'SetInvisible', 'SetHiddenCount', 'SetHiddenRotation', 'SetHiddenState', 'SetHiddenValue', 'SetHiddenWhat', 'SetHiddenWho']
ignore_null_columns = ['Behaviour', 'StateRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation', 'Clarity', 'Decisiveness', 'Drama', 'MoveEvaluation', 'StateEvaluationDifference', 'BoardSitesOccupied', 'BranchingFactor', 'DecisionFactor', 'MoveDistance', 'PieceNumber', 'ScoreDifference']
ignore_game_columns = ['GameRulesetName', 'EnglishRules', 'LudRules']
new_duplicates = ['AsymmetricForces','AsymmetricPiecesType','PieceDirection','Team','SpiralTiling','CircleTiling','ShibumiStyle','Sow','MancalaStyle','NumPerimeterSites','SetRotation','Roll','SwapOption','SetRotationFrequency','PathExtent','LoopLoss','PathExtentEnd','PathExtentWin','PathExtentLoss','SowOriginFirst','LoopEnd','TerritoryEnd','TerritoryWin','LeftwardDirection','LeftwardsDirection','ForwardRightDirection','BackwardRightDirection','LoopWinFrequency','PatternWin','PatternWinFrequency','TerritoryWinFrequency','NoProgressDrawFrequency','StackState']


train_df = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")

#### Add Augmented data
temp = train_df.copy()
temp['AdvantageP1'] = 1 - temp['AdvantageP1']
temp['agent1'], temp['agent2'] = temp['agent2'], temp['agent1']
temp['utility_agent1'] = -temp['utility_agent1']
train_df = pd.concat([train_df,temp]).reset_index(drop=True)
train_df = train_df.drop('Id',axis=1).drop_duplicates().reset_index(drop=True)

#### Get the groups
groups = train_df['GameRulesetName'].copy()
# groups = train_df['LudRules'].str.extract(r'\(game\s+"([^"]+)"')

train_df = train_df.drop(ignore_null_columns + ignore_constant_columns + ignore_game_columns + new_duplicates,axis=1)

# Add eight features extracted from player names
def extract_agent_info(df, agent_col, prefix):
    df[f'{prefix}_selection'] = df[agent_col].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[0]
    df[f'{prefix}_exploration'] = df[agent_col].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[1].astype(float)
    df[f'{prefix}_playout'] = df[agent_col].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[2]
    df[f'{prefix}_bounds'] = df[agent_col].str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', expand=True)[3]

extract_agent_info(train_df, 'agent1', 'p1')
extract_agent_info(train_df, 'agent2', 'p2')

train_df = reduce_memory_usage(train_df)


def apply_ensemble(submissions, weights, scales, constants):

    # Transform each submission and compute weighted sum
    transformed_subs = [(scale * sub + const) 
                       for sub, scale, const 
                       in zip(submissions, scales, constants)]
    
    # Compute weighted sum
    ensemble_prediction = sum(weight * sub 
                            for weight, sub 
                            in zip(weights, transformed_subs))
    
    return np.clip(ensemble_prediction, -0.9715,0.9649)

# Your optimized parameters
weights = [0.39416418, 0.24588782, 0.31496934, 0.04497866]
scales = [1.24084309, 1.18381245, 1.21937415, 1.0        ]
constants = [0.01421057, 0.01084297, 0.0128257,  0.0        ]

# subs

In [9]:
import pickle
from glob import glob
counter = 0  
def predict(test, submission):
    global counter 
    test_copy = test.clone()
    wts = [1.0,0.0,0.0]
    print(f"using weights {wts}")
    train_model(wts) if counter == 0 else None
    test = test.to_pandas()
    test_len = len(test)
    # TTA
    temp = test.copy()
    temp['AdvantageP1'] = 1 - temp['AdvantageP1']
    temp['agent1'], temp['agent2'] = temp['agent2'], temp['agent1']
    test = pd.concat([test,temp]).reset_index(drop=True)  
    test = pl.DataFrame(test)    
    counter += 1 
    test = fe.process_agent_cols(test) 
    test = fe.set_datatypes(test)  
    if CFG.tfidf:
        df = fe.do_tfidf(test,True)   
    print("Test shape ",test.shape)  
    
    if CFG.drop_gameruleset: 
        try:
            globals["drop_cols"] = list(globals["drop_cols"] +["GameRulesetName"])
        except BaseException as e:
            print("An exception occurred",e)  
    test, _ = fe.process_data(test, drop_cols +["GameRulesetName"] if CFG.drop_gameruleset else drop_cols, False) 
    tab_preds = 0
    predictions1 = md.inference(test,cat_cols,lgb_models,cat_models,None,test_len,wts)
    
    
    
    ## CB
    test = test_copy.to_pandas()
    test = test.drop(ignore_null_columns + ignore_constant_columns + ignore_game_columns + new_duplicates,axis=1)
    test_len = len(test)
    
    
    test = test.drop('Id',axis=1)
    extract_agent_info(test, 'agent1', 'p1')
    extract_agent_info(test, 'agent2', 'p2')
    test = reduce_memory_usage(test)
    
    # TTA
    temp = test.copy()
    temp['AdvantageP1'] = 1 - temp['AdvantageP1']
    temp['agent1'], temp['agent2'] = temp['agent2'], temp['agent1']
    temp["p1_selection"], temp["p2_selection"] = temp["p2_selection"], temp["p1_selection"] 
    temp["p1_exploration"], temp["p2_exploration"] = temp["p2_exploration"], temp["p1_exploration"] 
    temp["p1_playout"], temp["p2_playout"] = temp["p2_playout"], temp["p1_playout"] 
    temp["p1_bounds"], temp["p2_bounds"] = temp["p2_bounds"], temp["p1_bounds"] 
    test = pd.concat([test,temp]).reset_index(drop=True)  
    
    X = train_df.drop(['utility_agent1', 'num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1'], axis=1)
    y = train_df['utility_agent1'].values  
    
     # Label Encoder
    feats = ['p1_selection', 'p1_exploration',
           'p1_playout', 'p1_bounds', 'p2_selection', 'p2_exploration',
           'p2_playout', 'p2_bounds','agent1','agent2']
    le = LabelEncoder()
    for f in feats:
        le.fit(X[f])
        X[f] = le.transform(X[f])
        test[f] = le.transform(test[f])

    # Inference loop for test_df
    model_paths = ["/kaggle/input/um-ckpts/cb_fold_0.pkl","/kaggle/input/um-ckpts-2/cb_fold_0.pkl"]
    predictions = np.zeros((len(test), 3))
    # Load each saved model and predict
    for i, model_path in tqdm(enumerate(model_paths)):
        
        with open(model_path, 'rb') as file:
            model = pickle.load(file)
        
        predictions[:, i] = (model.predict(test)) + test['AdvantageP1']


    
    # Load each saved model and predict
    for model_path in glob("/kaggle/input/um-ckpts/lgbm_fold*"):
        with open(model_path, 'rb') as file:
            model = pickle.load(file)
        
        predictions[:, 2] = (model.predict(test[X.columns])) 
    
  
    ## Ensemble
    advp1_utility = -(test["AdvantageP1"] - (1 - test["AdvantageP1"])).values[:test_len].reshape(-1,)
    preds = apply_ensemble([predictions1,-predictions[test_len:,2] , predictions[:test_len,0], advp1_utility], weights, scales, constants)
    print(preds)
    return submission.with_columns(pl.Series('utility_agent1', preds)) 

In [10]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

using weights [1.0, 0.0, 0.0]
Processing raw training data...


/kaggle/input/mcts-gaurav-dart-tab//dart_5f_V8/fe.py:141: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns([pl.col(col).cast(pl.Categorical) for col in cat_cols if col in df.columns])


Calculating correlation removing more than 0.8
Variable dropped with more than 80.0% 77
Features selected for dropping: 388 
before removal (233234, 867)
after removal (233234, 849)
Shape: (233234, 401)
Memory usage: 356.81 MB

Processed data saved to /kaggle/input/mcts-gaurav-dart-tab//dart_5f_V8_processed_train_data.csv and cat_cols saved to /kaggle/input/mcts-gaurav-dart-tab//dart_5f_V8_cat_cols.json
#### Columns used  (233234, 401) ['GameRulesetName', 'agent1_1', 'agent1_2', 'agent1_3', 'agent1_4', 'agent2_1', 'agent2_2', 'agent2_3', 'agent2_4']
using existing fold indicies

 fold 0 catb CAT CV RMSE: 0.3779538279244822

 fold 1 catb CAT CV RMSE: 0.4255574828119897

 fold 2 catb CAT CV RMSE: 0.42624032122814326

 fold 3 catb CAT CV RMSE: 0.3920211943239598

 fold 4 catb CAT CV RMSE: 0.3963979069036515

 fold 5 catb CAT CV RMSE: 0.4384884910786434

 fold 6 catb CAT CV RMSE: 0.4617225671618552

 fold 7 catb CAT CV RMSE: 0.39971203196802163

 catb CAT CV RMSE: 0.4155835220680536
using 

####  Ensemble Score 0.4145693673983757 ####
Test shape  (6, 816)
An exception occurred 'builtin_function_or_method' object is not subscriptable
before removal (6, 863)
after removal (6, 863)
Shape: (6, 399)
Memory usage: 0.01 MB



2it [00:02,  1.33s/it]


[ 0.25245695  0.11935638 -0.24372749]
